# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.37,94,100,11.38,TF,1699563906
1,1,albany,42.6001,-73.9662,9.65,81,100,0.45,US,1699563326
2,2,soe,-9.8607,124.2840,19.05,71,8,1.37,ID,1699563907
3,3,vila velha,-20.3297,-40.2925,26.97,69,0,8.75,BR,1699563644
4,4,puerto natales,-51.7236,-72.4875,13.25,38,20,15.43,CL,1699563675


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    hover_cols = ["City", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=20) & (city_data_df["Max Temp"]<=30) \
                              & (city_data_df["Humidity"]<=50) & (city_data_df["Cloudiness"]<=26)]

# Drop any rows with null values
city_clean = city_ideal.dropna()

# Display sample data
city_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,port lincoln,-34.7333,135.8667,21.61,45,26,3.05,AU,1699563917
34,34,patti,31.2808,74.8572,21.04,49,18,2.65,IN,1699563919
57,57,san rafael,-34.6177,-68.3301,27.18,17,0,3.05,AR,1699563928
58,58,lompoc,34.6391,-120.4579,26.08,48,0,7.72,US,1699563670
70,70,ha'il,27.5219,41.6907,20.01,42,0,2.06,SA,1699563933
72,72,karratha,-20.7377,116.8463,27.19,31,0,5.43,AU,1699563796
87,87,fort bragg,35.1390,-79.0060,27.58,48,0,2.57,US,1699563771
96,96,beitbridge,-22.2167,30.0000,25.01,45,0,6.04,ZW,1699563941
108,108,taoudenni,22.6783,-3.9836,24.25,13,0,4.68,ML,1699563945
110,110,rakops,-21.0167,24.3333,29.42,27,0,1.98,BW,1699563946


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_clean[["City", "Country","Lat", "Lng", "Humidity" ]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\rgreg\AppData\Local\Temp\ipykernel_18904\852216884.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
26,port lincoln,AU,-34.7333,135.8667,45,
34,patti,IN,31.2808,74.8572,49,
57,san rafael,AR,-34.6177,-68.3301,17,
58,lompoc,US,34.6391,-120.4579,48,
70,ha'il,SA,27.5219,41.6907,42,
72,karratha,AU,-20.7377,116.8463,31,
87,fort bragg,US,35.1390,-79.0060,48,
96,beitbridge,ZW,-22.2167,30.0000,45,
108,taoudenni,ML,22.6783,-3.9836,13,
110,rakops,BW,-21.0167,24.3333,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories":"hotel",
    "apiKey": geoapify_key,
    "limit":10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port lincoln - nearest hotel: No hotel found
patti - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
beitbridge - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
rakops - nearest hotel: No hotel found
las varillas - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
ceres - nearest hotel: No hotel found
ingeniero jacobacci - nearest hotel: No hotel found
ramadi - nearest hotel: No hotel found
dongola - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
abeche - nearest hotel: No hotel found
ubari - nearest hotel: No hotel found
ar rutbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
26,port lincoln,AU,-34.7333,135.8667,45,No hotel found
34,patti,IN,31.2808,74.8572,49,No hotel found
57,san rafael,AR,-34.6177,-68.3301,17,No hotel found
58,lompoc,US,34.6391,-120.4579,48,No hotel found
70,ha'il,SA,27.5219,41.6907,42,No hotel found
72,karratha,AU,-20.7377,116.8463,31,No hotel found
87,fort bragg,US,35.1390,-79.0060,48,No hotel found
96,beitbridge,ZW,-22.2167,30.0000,45,No hotel found
108,taoudenni,ML,22.6783,-3.9836,13,No hotel found
110,rakops,BW,-21.0167,24.3333,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    hover_cols = ["City", "Humidity","Hotel", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)